In [5]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
sys.path.append('../')
import folps as FOLPS
from classy import Class
# from scipy.interpolate import RectBivariateSpline
# from scipy.interpolate import CubicSpline
# import os, h5py
# import time

!pwd

✅ Using NumPy with CPU.
/Users/waco/Desktop/all/FolpsDpipelines/folpsD/notebooks


### Load data

In [8]:
k_window = np.array([0.00574982, 0.01023557, 0.01526211, 0.02031268, 0.02516689,
       0.03020486, 0.03517139, 0.04012842, 0.04507161, 0.05004095,
       0.05507535, 0.0600698 , 0.06512571, 0.07011277, 0.07502157,
       0.08004289, 0.08504873, 0.09004322, 0.09502533, 0.1000098 ,
       0.10504149, 0.1100224 , 0.11502614, 0.12001702, 0.12502202,
       0.13002994, 0.13502229, 0.14005739, 0.14502593, 0.1500209 ,
       0.15502526, 0.160035  , 0.16505188, 0.17003695, 0.1750324 ,
       0.18003276, 0.18504025, 0.1900268 , 0.19500211, 0.20001755,
       0.20502544, 0.21001923, 0.21502351, 0.22001683, 0.22501584,
       0.23002834, 0.23502663, 0.2400277 , 0.24503438, 0.25002842,
       0.25501355, 0.26001043, 0.26502645, 0.27002398, 0.27502142,
       0.28002714, 0.28503308, 0.29001812, 0.29500021, 0.30001302,
       0.3050127 , 0.31002231, 0.31501079, 0.32000858, 0.32502457,
       0.33001103, 0.33501358, 0.34001106, 0.34501103, 0.35001633,
       0.35501422, 0.3600208 , 0.36501188, 0.37001315, 0.37501782,
       0.3800033 , 0.38501567, 0.39002782, 0.39500995, 0.40000344])

### run code

In [11]:
#LRG1 Abacus 2nd Gen best fit cosmology
X_FoG_b = 1
h=0.677
omega_cdm=0.122
omega_b=0.02236
logAs = 3.03
X_FoG_p   = 1.56
b1   = 1.85
b2   = 0.32 
bs=-1.4
# b2= 8/21*(b1-1)
# bs2= -4/7*(b1-1)      
b3nl = 32/315*(b1 - 1);  
c1=-4
c2=-0.1
Pshot=-1820
Bshot = 5115
omega_ncdm = 0.06/96.4
ns=0.966

z_pk = 0.5;                 #evaluation redshift
CosmoParams = [z_pk, omega_b, omega_cdm, omega_ncdm, h]

bisp_nuis_params = [b1, b2, bs, c1,c2,Pshot,Bshot, X_FoG_b]
OmM=(omega_cdm+omega_b+omega_ncdm)/h**2
bisp_cosmo_params = [z_pk,OmM,h]

qpar,qperp=1,1

In [13]:
cosmo = Class()
# pass input parameters
cosmo.set({'omega_b':omega_b,'omega_cdm':omega_cdm,'h':h,'A_s':np.exp(logAs)*1e-10,'n_s':ns})
cosmo.set({'output':',mPk', 
           'P_k_max_1/Mpc':10.0,
           'z_pk': z_pk})
# run class
cosmo.compute()  # output is in 1/Mpc

k = np.logspace(-4,np.log10(3),1000) # k in h/Mpc
Pk = [] # P(k) in (Mpc/h)**3
h = cosmo.h() # get reduced Hubble for conversions to 1/Mpc
for k_v in k:
    Pk.append(cosmo.pk(k_v*h,z_pk )*h**3) # pk(k,z) in h/Mpc

inputpkT = np.array([k, Pk])   

kwargs = {'z': z_pk,
          'h': h,
          'Omega_m': (omega_cdm+omega_b+omega_ncdm)/h**2,
          # 'f0': np.float64(0.6880638641959066),  #if not provided, the code computes it
          'fnu': 0.00}

f0 = FOLPS.f0_function(z_pk,OmM)
print(f0)

0.7620132852753934


In [16]:
NuisanParamsPk = [b1, b2, bs, b3nl, 0,0,0, 0,0,0, 0,0]
Omfid = -1  # for AP set > 0
APtrue = False
matrix = FOLPS.MatrixCalculator()
mmatrices = matrix.get_mmatrices()
k_ev = np.logspace(np.log10(0.01), np.log10(0.3), num = 100) # array of k_ev in [h/Mpc]
nonlinear = FOLPS.NonLinearPowerSpectrumCalculator(mmatrices=mmatrices,
                                             kernels='fk',
                                             **kwargs)

table, table_nonwiggles = nonlinear.calculate_loop_table(k=inputpkT[0], pklin=inputpkT[1],cosmo=None, **kwargs)

k_pkl_pklnw_=np.array([table[0], table[1], table_nonwiggles[1]])



Loading matrices from matrices_nfftlog128_Afull-True_use_TNS-False.npy


In [18]:
## if you dont want to run nonlinear.calculate_loop_table(...), which is slow. You can instead do the following 
linear=nonlinear.get_linear(inputpkT[0], inputpkT[1], pknow=None, cosmo=None, **kwargs)   
k_pkl_pklnw = np.array([linear['k'], linear['pk_l'],linear['pk_l_NW']])
#k_pkl_pklnw_-k_pkl_pklnw

In [22]:
%%time 

Ssize=20
bc = FOLPS.WindowConvolvedBispectrum(model="FOLPSD")

Bl1l2L = bc.reduced_Bl1l2L(bisp_nuis_params, bisp_cosmo_params, 
                     qpar, qperp, k_pkl_pklnw, k_window, Ssize,
                     precision_full=[8,10,10], precision_diag=[12,15,15],
                     f=f0, 
                     renormalize=True, 
                     interpolation_method_full='linear',
                     interpolation_method_diag='linear',
                     use_full_diag=True)


i, j = np.triu_indices(len(k_window))
coords = np.stack((k_window[i], k_window[j]), axis=1)
bk_000 = Bl1l2L[0][i,j]
bk_110 = Bl1l2L[1][i,j]
bk_220 = Bl1l2L[2][i,j]
bk_112 = Bl1l2L[3][i,j]
bk_202 = Bl1l2L[4][i,j]
B022 = Bl1l2L[4].T
bk_022 = B022[i,j]

CPU times: user 50.9 ms, sys: 16.5 ms, total: 67.3 ms
Wall time: 66.2 ms


In [24]:
#coords_000 = coords

In [26]:
coords.shape

(3240, 2)

In [30]:
bk_000.shape

(3240,)